In [17]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import operator
from analysis_count_0g import read_n, classify_imgs2
%matplotlib inline

In [5]:
model_name = "3task_0305"
num_runs = 1
test_trials = 10
iter_list = [250]#np.arange(0,3500,500)#[3200, 4800, 6400, 9600, 12800]#, 200, 300, 400, 600, 800, 1200]#, 1600, 2400, 3200, 4800, 6400, 9600, 12800, 19200, 25600, 38400, 51200, 76800, 102400]#, 153600, 204800, 307200, 409600, 614400, 819200, 1000000]
            #[0, 400, 800, 1600, 3200, 6400, 12800, 25600]#, 51200, 76800, 102400]#, 153600, 204800, 307200, 409600, 614400, 819200, 1000000, 2000000, 3000000, 4000000]
             #[0, 1000, 4000, 16000, 32000, 64000, 125000, 250000, 500000, 600000]
#iter_list = np.arange(0,11100000,100000)
glimpse_list = np.arange(0,10,1)
max_blobs = 9
min_blobs = 1

num_iters = len(iter_list)
num_glimpses = len(glimpse_list)
blob_list = np.arange(0,10,1)
output_size = max_blobs - min_blobs + 2 # 10
data_directory = "data/" + model_name + "/"
m = 0.5
num_imgs = test_trials*(output_size-1)

blob_point_all_runs = np.zeros([num_runs, num_iters, num_imgs, output_size+1])
class_all_runs = np.zeros([num_runs, num_iters, num_imgs])
pred_gx_all_runs = np.zeros([num_runs, num_iters, num_imgs, output_size+1])
tchr_gx_all_runs = np.zeros([num_runs, num_iters, num_imgs, output_size+1])
pred_gy_all_runs = np.zeros([num_runs, num_iters, num_imgs, output_size+1])
tchr_gy_all_runs = np.zeros([num_runs, num_iters, num_imgs, output_size+1])
size_all_runs = np.zeros([num_runs, num_iters, num_imgs, output_size+1])

In [6]:
def fill_matrix(path, iteration):
    """Fill the confidence and choice matrices for one run at one iteration."""
    
    data = None
    num_imgs = test_trials*(output_size-1) # batch_size
    imgs_data = classify_imgs2(iteration, True, num_imgs, path=path) # new_imgs = True
        
    blob_point_one_run = np.zeros([num_imgs, output_size+1])
    class_one_run = np.zeros(num_imgs)
    pred_gx_one_run = np.zeros([num_imgs, output_size+1])
    tchr_gx_one_run = np.zeros([num_imgs, output_size+1])
    pred_gy_one_run = np.zeros([num_imgs, output_size+1])
    tchr_gy_one_run = np.zeros([num_imgs, output_size+1])
    size_one_run = np.zeros([num_imgs, output_size+1])
    
    for idx, data in enumerate(imgs_data):
        for g in range(num_glimpses):
            blob_point_one_run[idx][g] = int(data["blob_point"][g])
            pred_gx_one_run[idx][g] = data["xs"][g][0]
            tchr_gx_one_run[idx][g] = data["xs"][g][1]
            pred_gy_one_run[idx][g] = data["ys"][g][0]
            tchr_gy_one_run[idx][g] = data["ys"][g][1]
            size_one_run[idx][g] = int(data["size"][g])
        class_one_run[idx] = data["class"]+1      
    
    return blob_point_one_run, class_one_run, pred_gx_one_run, tchr_gx_one_run, pred_gy_one_run, tchr_gy_one_run, size_one_run

In [7]:
for run in range(num_runs):
    path = 'model_runs/' + model_name + '_run_' + str(run + 1) 
#     path = 'model_runs/DRAM_test_square'
    for i, iteration in enumerate(iter_list):
        blob_point_all_runs[run, i], class_all_runs[run, i], pred_gx_all_runs[run, i], tchr_gx_all_runs[run, i], pred_gy_all_runs[run, i], tchr_gy_all_runs[run, i], size_all_runs[run, i] = fill_matrix(path, iteration)

INFO:tensorflow:Restoring parameters from model_runs/3task_0305_run_1/countmodel_250.ckpt


In [5]:
class_all_runs[0,1,2]
tchr_gx_all_runs[0,0,0][0:10]
#size_all_runs[0,0,0]
blob_point_all_runs[0,1,0]
tchr_gx_all_runs[0,1,0][0:10]

array([  9.,  18.,  27.,  35.,  35.,  35.,  35.,  35.,  35.,  35.])

In [6]:
tchr_gx_all_runs[0,1,0][0:10]

array([  9.,  18.,  27.,  35.,  35.,  35.,  35.,  35.,  35.,  35.])

In [13]:
import sys
import numpy as np
import random

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.io import output_notebook

it=0
for t in range(num_imgs):
    if class_all_runs[0,it,t]==5:
        img=t
        break
        
# parameters
N = 15  # N x N filterbank for each layer
dims = [100,100] # input_img size

# generate blobs
width = 5
height = 5

# top left corner coordinate
cX = tchr_gx_all_runs[0,it,img][0:int(class_all_runs[0,it,img])]-2.5
cY = tchr_gy_all_runs[0,it,img][0:int(class_all_runs[0,it,img])]-2.5
#cX = [8,17,25,33,42,51,60,68,77]
#cY = [47,48,48,47,48,47,47,48,47]

k = len(cX)

# input_img
input_img = np.ones(dims[0]*dims[1])
for i in range(k):
    for j in range(int(cY[i]), int(cY[i]+height)):
        for k in range(int(cX[i]), int(cX[i]+width)):
            input_img[j*dims[0]+k] = 255
            
input_img = np.reshape(input_img,(dims[0],dims[1]))

In [ ]:
tchr_gx_all_runs[0,0,0]

In [16]:
print("blob_point:", blob_point_all_runs[0,it,img][0:int(class_all_runs[0,it,img]+1)])
for i in range(int(class_all_runs[0,it,img]+1)):
               # attention window
               gx = tchr_gx_all_runs[0,it,img,i]
               gy = tchr_gy_all_runs[0,it,img,i]
               pred_gx = pred_gx_all_runs[0,it,img,i]
               pred_gy = pred_gy_all_runs[0,it,img,i]
               print("gx:", gx)
               print("gy:", gy)
               xx=np.reshape([gx-3.5,gx,gx+3.5], (-1,1))
               yy=np.reshape([gy-3.5,gy,gy+3.5], (-1,1))
               mu_x_1_list = np.reshape([xx]*3,[-1])
               mu_y_1_list = np.reshape(np.transpose([yy]*3),[-1])

               input_img = np.reshape(input_img,(dims[0],dims[1]))

               cross_source = ColumnDataSource(
                   data=dict(
                       cross_x=[gx],
                       cross_y=[gy],
                       desc=['Att_window Center'],
                   )
               )

               dot_source_1 = ColumnDataSource(
                   data=dict(
                       mu_x_1=mu_x_1_list,
                       mu_y_1=mu_y_1_list,
                   )
               )

               source = ColumnDataSource(
                   data=dict(
                       x=[pred_gx],
                       y=[pred_gy],
                   )
               )

               p = figure(title="Input Image (gx=%-d, gy=%-d)" %(gx,gy), x_range=(0, 100), y_range=(0, 100))
               iii = p.image(image=[input_img], x=0, y=0, dw=100, dh=100, palette="Greys256")
               p.circle('mu_x_1', 'mu_y_1', size=5, source=dot_source_1, color="orange")
               p.cross('cross_x', 'cross_y', size=10, source=cross_source, color="blue")
               p.circle('x', 'y', size=5, source=source, color="red")

               output_notebook()

               show(p)

blob_point: [ 1.  2. -1.  3.  3.  3.]
gx: 11.5
gy: 49.5


Loading BokehJS ...

gx: 19.5
gy: 50.5


Loading BokehJS ...

gx: 28.5
gy: 49.5


...

Loading BokehJS ...

gx: 45.5
gy: 49.5


Loading BokehJS ...

gx: 45.0
gy: 49.0


Loading BokehJS ...

In [15]:
gx

45.0

# Generate Image

In [66]:
import sys
import numpy as np
import random

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.io import output_notebook

# parameters
N = 15  # N x N filterbank for each layer
dims = [100,100] # input_img size
num_filter = N*N+N*N
# delta, sigma2
delta_1=max(dims[0],dims[1])*1.5/(N-1)
sigma2_1=delta_1*delta_1/4 # sigma=delta/2
delta_2=max(dims[0],dims[1])/3/(N-1)
sigma2_2=delta_2*delta_2/4 # sigma=delta/2

num_blobs = 5
img = np.ones(dims[0]*dims[1])
img_width = dims[0]
img_height = dims[1]
num_count = 0
glimpse_count = 0
# top left corner coordinate
cX_prev = 0
cY_prev = 0
# generate blobs
while num_count < num_blobs:
    width = 5
    height = 5
    cX_ = random.randint(8, 9)
    cY_ = 3
    if num_count == 0:
        cX = cX_
        cY = random.randint(img_height/2 - cY_, img_height/2 + cY_ - height)
        cX0 = cX
        cY0 = cY
    else:
        cX = cX_prev + cX_
        cY = random.randint(img_height/2 - cY_, img_height/2 + cY_ - height)
    cX_prev = cX
    cY_prev = cY
        
    for p in range(cY, cY+height):
        for q in range(cX, cX+width):
            img[p*img_width+q] = 255
                
    num_count += 1
    glimpse_count += 1

# attention window
gx = cX0+2.5#dims[0]/2
gy = cY0+2.5#dims[1]/2

grid_i = np.arange(N)
mu_x_1 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_1, (-1,1))
mu_y_1 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_1, (-1,1)) 
mu_x_2 = np.reshape(gx + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
mu_y_2 = np.reshape(gy + (grid_i - N / 2 + 0.5) * delta_2, (-1,1))
a = np.reshape(np.arange(dims[0]), (1, -1)) 
b = np.reshape(np.arange(dims[1]), (1, -1)) 

mu_x_1_list = np.reshape([mu_x_1]*N,[-1]) 
mu_y_1_list = np.reshape(np.transpose([mu_y_1]*N),[-1]) 
mu_x_2_list = np.reshape([mu_x_2]*N,[-1]) 
mu_y_2_list = np.reshape(np.transpose([mu_y_2]*N),[-1])

Fx_1 = np.exp(-np.square(a - mu_x_1) / (2*sigma2_1)) 
Fy_1 = np.exp(-np.square(b - mu_y_1) / (2*sigma2_1)) 
Fx_2 = np.exp(-np.square(a - mu_x_2) / (2*sigma2_2))
Fy_2 = np.exp(-np.square(b - mu_y_2) / (2*sigma2_2))

# normalize, sum over A and B dims
Fx_1=Fx_1/np.reshape(np.sum(Fx_1,axis=1),(-1,1))
Fy_1=Fy_1/np.reshape(np.sum(Fy_1,axis=1),(-1,1))
Fx_2=Fx_2/np.reshape(np.sum(Fx_2,axis=1),(-1,1))
Fy_2=Fy_2/np.reshape(np.sum(Fy_2,axis=1),(-1,1))

In [76]:
input_img = np.reshape(img,(dims[0],dims[1]))
#filter_img
Fxt_1=np.transpose(Fx_1)
np.shape(Fxt_1)
filter_img_1=np.matmul(Fy_1, np.matmul(input_img, Fxt_1))
filter_img_1=np.log(np.reshape(filter_img_1,(N,N)))

Fxt_2=np.transpose(Fx_2)
np.shape(Fxt_2)
filter_img_2=np.matmul(Fy_2, np.matmul(input_img, Fxt_2))
filter_img_2=np.log(np.reshape(filter_img_2,(N,N)))

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <span style="font-size: 15px; font-weight: bold;">@desc</span>
            </div>
            <div>
                <span style="font-size: 12px;">Location</span>
                <span style="font-size: 12px; color: #320;">($x, $y)</span>
            </div>
        </div>
        """
    )


source = ColumnDataSource(
        data=dict(
            x=[gx],
            y=[gy],
            desc=['Att_window Center'],
        )
)

dot_source_1 = ColumnDataSource(
        data=dict(
            mu_x_1=mu_x_1_list,
            mu_y_1=mu_y_1_list,
            #desc=['Att_window Center'],
        )
)

dot_source_2 = ColumnDataSource(
        data=dict(
            mu_x_2=mu_x_2_list,
            mu_y_2=mu_y_2_list,
            #desc=['Att_window Center'],
        )
)

p = figure(title="Input Image (gx=%-d, gy=%-d)" %(gx,gy), x_range=(0, 20), y_range=(40, 60), tools=[hover])
iii = p.image(image=[input_img], x=0, y=0, dw=100, dh=100, palette="Greys256")
p.circle('mu_x_1', 'mu_y_1', size=5, source=dot_source_1, color="orange")
p.circle('mu_x_2', 'mu_y_2', size=5, source=dot_source_2, color="blue")
p.circle('x', 'y', size=3, source=source, color="red")



filter_source = ColumnDataSource(
        data=dict(
            x=[N/2],
            y=[N/2],
            desc=['Filterbank Center'],
        )
)


p_filter_1 = figure(title="Outer Layer Filter Image", x_range=(0, 15), y_range=(0, 15))
iii = p_filter_1.image(image=[filter_img_1], x=0, y=0, dw=15, dh=15, palette="Greys256")
p_filter_1.circle('x', 'y', size=10, source=filter_source, color="red")

p_filter_2 = figure(title="Inner Layer Filter Image", x_range=(0, 15), y_range=(0, 15))
iii = p_filter_2.image(image=[filter_img_2], x=0, y=0, dw=15, dh=15, palette="Greys256")
p_filter_2.circle('x', 'y', size=10, source=filter_source, color="red")

output_notebook()

show(p)
pp = gridplot([[p_filter_1,p_filter_2]], toolbar_location="right")
show(pp)

Loading BokehJS ...

In [54]:
import sys
import numpy as np
import random

from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.io import output_notebook

# parameters
N = 15  # N x N filterbank for each layer
dims = [100,100] # input_img size
num_blobs = 5
img = np.ones(dims[0]*dims[1])
img_width = dims[0]
img_height = dims[1]
num_count = 0
glimpse_count = 0
# generate blobs
for p in range(10, 10+6):
    for q in range(10, 10+6):
        img[p*img_width+q] = 255
        
input_img = np.reshape(img,(dims[0],dims[1]))
p = figure(x_range=(-10, 110), y_range=(-10, 110))
iii = p.image(image=[input_img], x=0, y=0, dw=100, dh=100, palette="Greys256")
output_notebook()
show(p)

Loading BokehJS ...

In [48]:
gx

10.5

In [49]:
gy

49.5

In [77]:
mu_x_2

array([[ -6.16666667],
       [ -3.78571429],
       [ -1.4047619 ],
       [  0.97619048],
       [  3.35714286],
       [  5.73809524],
       [  8.11904762],
       [ 10.5       ],
       [ 12.88095238],
       [ 15.26190476],
       [ 17.64285714],
       [ 20.02380952],
       [ 22.4047619 ],
       [ 24.78571429],
       [ 27.16666667]])

In [51]:
cX0

8

In [52]:
cY0

47